In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [32]:
import pandas as pd
import numpy as np
import keras
import warnings
warnings.filterwarnings(action="ignore")
import matplotlib.pyplot as plt
import os
%matplotlib inline
from keras import backend as K
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Flatten
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import sklearn.metrics as metrics

In [33]:
import os
import pandas as pd

bacterial_dir = r'/kaggle/input/bacterial-vs-viral-largedataset/Curated X-Ray Dataset/Pneumonia-Bacterial'
viral_dir = r'/kaggle/input/bacterial-vs-viral-largedataset/Curated X-Ray Dataset/Pneumonia-Viral'

filepaths = []
labels = []
dict_lists = [os.listdir(bacterial_dir), os.listdir(viral_dir)]  # List of directory contents

class_labels = ['Bacterial', 'Viral']

for i, dir_list in enumerate(dict_lists):
    for f in dir_list:
        fpath = os.path.join(bacterial_dir if i == 0 else viral_dir, f)  # Construct full file path
        filepaths.append(fpath)
        labels.append(class_labels[i])

Fseries = pd.Series(filepaths, name="filepaths")
Lseries = pd.Series(labels, name="labels")
pneumonia_type_data = pd.concat([Fseries, Lseries], axis=1)
pneumonia_type_df = pd.DataFrame(pneumonia_type_data)
print(pneumonia_type_df.head())
print(pneumonia_type_df["labels"].value_counts())

                                           filepaths     labels
0  /kaggle/input/bacterial-vs-viral-largedataset/...  Bacterial
1  /kaggle/input/bacterial-vs-viral-largedataset/...  Bacterial
2  /kaggle/input/bacterial-vs-viral-largedataset/...  Bacterial
3  /kaggle/input/bacterial-vs-viral-largedataset/...  Bacterial
4  /kaggle/input/bacterial-vs-viral-largedataset/...  Bacterial
labels
Bacterial    5781
Viral        3149
Name: count, dtype: int64


In [34]:
train_images, test_images = train_test_split(pneumonia_type_df, test_size=0.2, random_state=42)
train_set, val_set = train_test_split(train_images, test_size=0.2, random_state=42)

In [35]:
image_gen = ImageDataGenerator(preprocessing_function= tf.keras.applications.mobilenet_v2.preprocess_input)

train = image_gen.flow_from_dataframe(dataframe= train_set,x_col="filepaths",y_col="labels",
                                      target_size=(224,224),
                                      color_mode='rgb',
                                      class_mode="categorical", #used for Sequential Model
                                      batch_size=4,
                                      shuffle=False            #do not shuffle data
                                     )
test = image_gen.flow_from_dataframe(dataframe= test_images,x_col="filepaths", y_col="labels",
                                     target_size=(224,224),
                                     color_mode='rgb',
                                     class_mode="categorical",
                                     batch_size=4,
                                     shuffle= False
                                    )
val = image_gen.flow_from_dataframe(dataframe= val_set,x_col="filepaths", y_col="labels",
                                    target_size=(224,224),
                                    color_mode= 'rgb',
                                    class_mode="categorical",
                                    batch_size=4,
                                    shuffle=False
                                   )

Found 5715 validated image filenames belonging to 2 classes.
Found 1786 validated image filenames belonging to 2 classes.
Found 1429 validated image filenames belonging to 2 classes.


## **Xception**

In [90]:
from keras.applications import Xception
from keras.models import Sequential
from keras.layers import GlobalAveragePooling2D, Dense, Dropout
from keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [91]:
img_width, img_height = 224, 224
input_shape = (img_width, img_height, 3)
xc_base_model = Xception(weights='imagenet', include_top=False, input_shape=input_shape)

83683744/83683744 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [92]:
for layer in xc_base_model.layers:
    layer.trainable = False

In [94]:
from tensorflow.keras.applications import ResNet50, Xception
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.models import Model

def create_model(model_name):
    if model_name == 'COVID-Net':
        base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
    elif model_name == 'ResNet':
        base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
    elif model_name == 'Xception':
        base_model = Xception(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
    elif model_name == 'EfficientNet':
        from efficientnet.keras import EfficientNetB0  # Import the EfficientNetB0 model
        base_model = EfficientNetB0(weights='imagenet', include_top=False, input_shape=(224,224,3))
    else:
        raise ValueError("Invalid model name. Choose from: 'COVID-Net', 'ResNet', 'InceptionV3'")

    # Freeze the base model layers
    for layer in base_model.layers:
        layer.trainable = False

    # Add a new classification head
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(512, activation='relu')(x)
    x = Dropout(0.5)(x)  # Apply dropout to the output tensor from the previous layer
    x = Dense(2, activation='softmax')(x)  # Binary classification output with softmax

    model = Model(inputs=base_model.input, outputs=x)
    model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

    return model
model=create_model('Xception')

In [96]:
#xception
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
learning_rate_reduction = ReduceLROnPlateau(monitor='val_accuracy', patience = 2, verbose=1,factor=0.3, min_lr=0.000001)
es = EarlyStopping(monitor='val_accuracy', patience=4, restore_best_weights=True)
history=model.fit(train, epochs=30, validation_data=val)

Epoch 1/30
   5/1429 ━━━━━━━━━━━━━━━━━━━━ 59s 42ms/step - accuracy: 0.5225 - loss: 0.7527

W0000 00:00:1714331425.125854     106 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


 189/1429 ━━━━━━━━━━━━━━━━━━━━ 1:19 64ms/step - accuracy: 0.6240 - loss: 0.6962

W0000 00:00:1714331436.968713     105 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


1428/1429 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step - accuracy: 0.6989 - loss: 0.6061

W0000 00:00:1714331479.993136     106 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


1429/1429 ━━━━━━━━━━━━━━━━━━━━ 78s 48ms/step - accuracy: 0.6990 - loss: 0.6060 - val_accuracy: 0.7579 - val_loss: 0.5391
Epoch 2/30
1429/1429 ━━━━━━━━━━━━━━━━━━━━ 53s 37ms/step - accuracy: 0.7615 - loss: 0.5121 - val_accuracy: 0.7628 - val_loss: 0.4983
Epoch 3/30
1429/1429 ━━━━━━━━━━━━━━━━━━━━ 61s 43ms/step - accuracy: 0.7757 - loss: 0.4816 - val_accuracy: 0.7712 - val_loss: 0.4962
Epoch 4/30
1429/1429 ━━━━━━━━━━━━━━━━━━━━ 53s 37ms/step - accuracy: 0.7781 - loss: 0.4767 - val_accuracy: 0.7894 - val_loss: 0.4848
Epoch 5/30
1429/1429 ━━━━━━━━━━━━━━━━━━━━ 53s 37ms/step - accuracy: 0.7865 - loss: 0.4620 - val_accuracy: 0.7656 - val_loss: 0.4835
Epoch 6/30
1429/1429 ━━━━━━━━━━━━━━━━━━━━ 53s 37ms/step - accuracy: 0.7976 - loss: 0.4527 - val_accuracy: 0.7810 - val_loss: 0.4743
Epoch 7/30
1429/1429 ━━━━━━━━━━━━━━━━━━━━ 54s 38ms/step - accuracy: 0.8115 - loss: 0.4283 - val_accuracy: 0.7894 - val_loss: 0.4512
Epoch 8/30
1429/1429 ━━━━━━━━━━━━━━━━━━━━ 54s 38ms/step - accuracy: 0.8053 - loss: 0.41

In [97]:
y_pred_xception = model.predict(test)

  6/447 ━━━━━━━━━━━━━━━━━━━━ 11s 26ms/step

W0000 00:00:1714333055.767854     105 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


447/447 ━━━━━━━━━━━━━━━━━━━━ 22s 42ms/step


W0000 00:00:1714333074.617978     105 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


In [98]:
pred_xception = np.argmax(y_pred_xception, axis=1) #pick class with highest  probability
labels = (train.class_indices)
labels = dict((v,k) for k,v in labels.items())
pred2_xception = [labels[k] for k in pred_xception]

In [99]:
from sklearn.metrics import confusion_matrix, accuracy_score
y_test = test_images.labels
print(classification_report(y_test, pred2_xception))
print("Accuracy of the Model:","{:.1f}%".format(accuracy_score(y_test, pred2_xception)*100))

              precision    recall  f1-score   support

   Bacterial       0.92      0.91      0.91      1146
       Viral       0.84      0.86      0.85       640

    accuracy                           0.89      1786
   macro avg       0.88      0.88      0.88      1786
weighted avg       0.89      0.89      0.89      1786

Accuracy of the Model: 88.9%


In [103]:
model.save("xception_model.h5")

## **DenseNet**

In [138]:
import tensorflow as tf
import keras
from tensorflow.keras.applications.densenet import DenseNet121
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.applications.inception_v3 import InceptionV3
from keras.applications import Xception
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D

In [139]:
from tensorflow.keras.applications import ResNet50, Xception
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.models import Model
from keras.applications import MobileNet
from keras.applications import InceptionV3

def create_model(model_name):
    if model_name == 'Dense-Net':
        base_model = DenseNet121(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
    elif model_name == 'MobileNet':
        base_model = MobileNet(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
    elif model_name == 'ResNet':
        base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
    elif model_name == 'Xception':
        base_model = Xception(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
    elif model_name == "InceptionV3":
        base_model = InceptionV3(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
    elif model_name == 'EfficientNet':
        from efficientnet.keras import EfficientNetB0  # Import the EfficientNetB0 model
        base_model = EfficientNetB0(weights='imagenet', include_top=False, input_shape=(224,224,3))
    else:
        raise ValueError("Invalid model name. Choose from: 'COVID-Net', 'ResNet', 'InceptionV3'")

    # Freeze the base model layers
    for layer in base_model.layers:
        layer.trainable = False

    # Add a new classification head
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(512, activation='relu')(x)
    x = Dense(512, activation='relu')(x)
    x = Dropout(0.5)(x)  # Apply dropout to the output tensor from the previous layer
    x = Dense(2, activation='softmax')(x)  # Binary classification output with softmax

    model = Model(inputs=base_model.input, outputs=x)
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

    return model

In [140]:
model_2=create_model('Dense-Net')

29084464/29084464 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [141]:
#densenet
from tensorflow.keras.callbacks import EarlyStopping

#es = EarlyStopping(monitor='val_accuracy', patience=4, restore_best_weights=True)
es = EarlyStopping(monitor='val_accuracy', patience=4, restore_best_weights=True)
history_2=model_2.fit(train, epochs=30, validation_data=val)

Epoch 1/30
 124/1429 ━━━━━━━━━━━━━━━━━━━━ 3:17 152ms/step - accuracy: 0.5861 - loss: 1.0292

W0000 00:00:1714335050.464580     106 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


1429/1429 ━━━━━━━━━━━━━━━━━━━━ 121s 64ms/step - accuracy: 0.6808 - loss: 0.6818 - val_accuracy: 0.7593 - val_loss: 0.5267
Epoch 2/30
1429/1429 ━━━━━━━━━━━━━━━━━━━━ 54s 38ms/step - accuracy: 0.7646 - loss: 0.5187 - val_accuracy: 0.7705 - val_loss: 0.4868
Epoch 3/30
1429/1429 ━━━━━━━━━━━━━━━━━━━━ 55s 38ms/step - accuracy: 0.7750 - loss: 0.4927 - val_accuracy: 0.7663 - val_loss: 0.4873
Epoch 4/30
1429/1429 ━━━━━━━━━━━━━━━━━━━━ 54s 38ms/step - accuracy: 0.7795 - loss: 0.4840 - val_accuracy: 0.7747 - val_loss: 0.4909
Epoch 5/30
1429/1429 ━━━━━━━━━━━━━━━━━━━━ 54s 38ms/step - accuracy: 0.7813 - loss: 0.4868 - val_accuracy: 0.7782 - val_loss: 0.4817
Epoch 6/30
1429/1429 ━━━━━━━━━━━━━━━━━━━━ 56s 39ms/step - accuracy: 0.7865 - loss: 0.4658 - val_accuracy: 0.7698 - val_loss: 0.4715
Epoch 7/30
1429/1429 ━━━━━━━━━━━━━━━━━━━━ 55s 39ms/step - accuracy: 0.7979 - loss: 0.4466 - val_accuracy: 0.7670 - val_loss: 0.5707
Epoch 8/30
1429/1429 ━━━━━━━━━━━━━━━━━━━━ 54s 38ms/step - accuracy: 0.7934 - loss: 0.4

KeyboardInterrupt: 

In [19]:
y_pred_densenet = model_2.predict(test)

  7/447 ━━━━━━━━━━━━━━━━━━━━ 14s 32ms/step

W0000 00:00:1714287744.459455     153 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


447/447 ━━━━━━━━━━━━━━━━━━━━ 43s 74ms/step


In [20]:
pred_densenet = np.argmax(y_pred_densenet, axis=1) #pick class with highest  probability
labels = (train.class_indices)
labels = dict((v,k) for k,v in labels.items())
pred2_densenet = [labels[k] for k in pred_densenet]

In [21]:
from sklearn.metrics import confusion_matrix, accuracy_score
y_test = test_images.labels
print(classification_report(y_test, pred2_densenet))
print("Accuracy of the Model:","{:.1f}%".format(accuracy_score(y_test, pred2_densenet)*100))

              precision    recall  f1-score   support

   Bacterial       0.78      0.90      0.83      1146
       Viral       0.75      0.55      0.63       640

    accuracy                           0.77      1786
   macro avg       0.76      0.72      0.73      1786
weighted avg       0.77      0.77      0.76      1786

Accuracy of the Model: 77.3%


## **VGG-16**

In [113]:
from tensorflow.keras.applications.vgg16 import VGG16

def create_vgg16_model():
    # Load the pre-trained VGG16 model
    base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

    # Freeze the base model layers
    for layer in base_model.layers:
        layer.trainable = False

    # Add a new classification head
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(2, activation='softmax')(x)

    model = Model(inputs=base_model.input, outputs=x)
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

    return model

In [114]:
vgg16_model = create_vgg16_model()

58889256/58889256 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [116]:
from tensorflow.keras.callbacks import EarlyStopping
es = EarlyStopping(monitor='val_accuracy', patience=4, restore_best_weights=True)
history_3=vgg16_model.fit(train, epochs=30, validation_data=val, callbacks=[es])

Epoch 1/30
1429/1429 ━━━━━━━━━━━━━━━━━━━━ 53s 37ms/step - accuracy: 0.7640 - loss: 0.5075 - val_accuracy: 0.7635 - val_loss: 0.5010
Epoch 2/30
1429/1429 ━━━━━━━━━━━━━━━━━━━━ 53s 37ms/step - accuracy: 0.7759 - loss: 0.4915 - val_accuracy: 0.7747 - val_loss: 0.4922
Epoch 3/30
1429/1429 ━━━━━━━━━━━━━━━━━━━━ 54s 37ms/step - accuracy: 0.7673 - loss: 0.4991 - val_accuracy: 0.7593 - val_loss: 0.5080
Epoch 4/30
1429/1429 ━━━━━━━━━━━━━━━━━━━━ 54s 38ms/step - accuracy: 0.7768 - loss: 0.4943 - val_accuracy: 0.7768 - val_loss: 0.4883
Epoch 5/30
1429/1429 ━━━━━━━━━━━━━━━━━━━━ 53s 37ms/step - accuracy: 0.7811 - loss: 0.4743 - val_accuracy: 0.7761 - val_loss: 0.4889
Epoch 6/30
1429/1429 ━━━━━━━━━━━━━━━━━━━━ 53s 37ms/step - accuracy: 0.7886 - loss: 0.4719 - val_accuracy: 0.7761 - val_loss: 0.4859
Epoch 7/30
1429/1429 ━━━━━━━━━━━━━━━━━━━━ 60s 42ms/step - accuracy: 0.7855 - loss: 0.4761 - val_accuracy: 0.7796 - val_loss: 0.4851
Epoch 8/30
1429/1429 ━━━━━━━━━━━━━━━━━━━━ 55s 38ms/step - accuracy: 0.7953 -

In [117]:
y_pred_vgg = vgg16_model.predict(test)

447/447 ━━━━━━━━━━━━━━━━━━━━ 19s 41ms/step


In [118]:
pred_vgg = np.argmax(y_pred_vgg, axis=1) #pick class with highest  probability
labels = (train.class_indices)
labels = dict((v,k) for k,v in labels.items())
pred2_vgg = [labels[k] for k in pred_vgg]

In [119]:
from sklearn.metrics import confusion_matrix, accuracy_score
y_test = test_images.labels
print(classification_report(y_test, pred2_vgg))
print("Accuracy of the Model:","{:.1f}%".format(accuracy_score(y_test, pred2_vgg)*100))

              precision    recall  f1-score   support

   Bacterial       0.79      0.87      0.83      1146
       Viral       0.72      0.59      0.65       640

    accuracy                           0.77      1786
   macro avg       0.75      0.73      0.74      1786
weighted avg       0.77      0.77      0.76      1786

Accuracy of the Model: 77.0%


## **MobileNet**

In [40]:
model_6=create_model('MobileNet')

17225924/17225924 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [41]:
#normal mobilenet
from tensorflow.keras.callbacks import EarlyStopping

#es = EarlyStopping(monitor='val_accuracy', patience=4, restore_best_weights=True)
es = EarlyStopping(monitor='val_accuracy', patience=4, restore_best_weights=True)
history_6=model_6.fit(train, epochs=30, validation_data=val)

Epoch 1/30
 951/1429 ━━━━━━━━━━━━━━━━━━━━ 15s 33ms/step - accuracy: 0.6778 - loss: 0.7859

W0000 00:00:1714328659.277934     105 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


1429/1429 ━━━━━━━━━━━━━━━━━━━━ 67s 42ms/step - accuracy: 0.6919 - loss: 0.7235 - val_accuracy: 0.7649 - val_loss: 0.4901
Epoch 2/30
1429/1429 ━━━━━━━━━━━━━━━━━━━━ 51s 36ms/step - accuracy: 0.7702 - loss: 0.4955 - val_accuracy: 0.7768 - val_loss: 0.4919
Epoch 3/30
1429/1429 ━━━━━━━━━━━━━━━━━━━━ 52s 36ms/step - accuracy: 0.7817 - loss: 0.4744 - val_accuracy: 0.7859 - val_loss: 0.4606
Epoch 4/30
1429/1429 ━━━━━━━━━━━━━━━━━━━━ 51s 36ms/step - accuracy: 0.7959 - loss: 0.4467 - val_accuracy: 0.7936 - val_loss: 0.4632
Epoch 5/30
1429/1429 ━━━━━━━━━━━━━━━━━━━━ 52s 36ms/step - accuracy: 0.8143 - loss: 0.4132 - val_accuracy: 0.8020 - val_loss: 0.4293
Epoch 6/30
1429/1429 ━━━━━━━━━━━━━━━━━━━━ 82s 36ms/step - accuracy: 0.8193 - loss: 0.4034 - val_accuracy: 0.8034 - val_loss: 0.4413
Epoch 7/30
1429/1429 ━━━━━━━━━━━━━━━━━━━━ 51s 35ms/step - accuracy: 0.8251 - loss: 0.3953 - val_accuracy: 0.8097 - val_loss: 0.4523
Epoch 8/30
1429/1429 ━━━━━━━━━━━━━━━━━━━━ 51s 36ms/step - accuracy: 0.8377 - loss: 0.37

In [45]:
y_pred_mobilenet = model_6.predict(test)

447/447 ━━━━━━━━━━━━━━━━━━━━ 12s 28ms/step


In [46]:
pred_mobilenet = np.argmax(y_pred_mobilenet, axis=1) #pick class with highest  probability
labels = (train.class_indices)
labels = dict((v,k) for k,v in labels.items())
pred2_mobilenet = [labels[k] for k in pred_mobilenet]

In [47]:
from sklearn.metrics import confusion_matrix, accuracy_score
y_test = test_images.labels
print(classification_report(y_test, pred2_mobilenet))
print("Accuracy of the Model:","{:.1f}%".format(accuracy_score(y_test, pred2_mobilenet)*100))

              precision    recall  f1-score   support

   Bacterial       0.84      0.92      0.88      1146
       Viral       0.83      0.69      0.75       640

    accuracy                           0.84      1786
   macro avg       0.83      0.80      0.81      1786
weighted avg       0.84      0.84      0.83      1786

Accuracy of the Model: 83.7%


In [74]:
model_6.save("mobilenet_model.h5")

## **Custom CNN**

In [36]:
custom_model = keras.models.Sequential([
    keras.layers.Conv2D(filters=128, kernel_size=(8, 8), strides=(3, 3), activation='relu', input_shape=(224, 224, 3)),
    keras.layers.BatchNormalization(),
    
    keras.layers.Conv2D(filters=256, kernel_size=(5, 5), strides=(1, 1), activation='relu', padding="same"),
    keras.layers.BatchNormalization(),
    keras.layers.MaxPool2D(pool_size=(3, 3)),
    
    keras.layers.Conv2D(filters=256, kernel_size=(3, 3), strides=(1, 1), activation='relu', padding="same"),
    keras.layers.BatchNormalization(),
    keras.layers.Conv2D(filters=256, kernel_size=(1, 1), strides=(1, 1), activation='relu', padding="same"),
    keras.layers.BatchNormalization(),
    keras.layers.Conv2D(filters=256, kernel_size=(1, 1), strides=(1, 1), activation='relu', padding="same"),
    keras.layers.BatchNormalization(),
    
    keras.layers.Conv2D(filters=512, kernel_size=(3, 3), activation='relu', padding="same"),
    keras.layers.BatchNormalization(),
    keras.layers.MaxPool2D(pool_size=(2, 2)),
    
    keras.layers.Conv2D(filters=512, kernel_size=(3, 3), activation='relu', padding="same"),
    keras.layers.BatchNormalization(),
    
    keras.layers.Conv2D(filters=512, kernel_size=(3, 3), activation='relu', padding="same"),
    keras.layers.BatchNormalization(),
    
    keras.layers.MaxPool2D(pool_size=(2, 2)),
    
    keras.layers.Conv2D(filters=512, kernel_size=(3, 3), activation='relu', padding="same"),
    keras.layers.BatchNormalization(),
    
    keras.layers.MaxPool2D(pool_size=(2, 2)),
    
    keras.layers.Flatten(),
    keras.layers.Dense(1024, activation='relu'),
#     keras.layers.Dropout(0.5),
    keras.layers.Dense(1024, activation='relu'),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(2, activation='softmax')
])

custom_model.compile(
    loss='categorical_crossentropy',
    optimizer=tf.optimizers.SGD(learning_rate=0.0001),
    metrics=['accuracy']
)

custom_model.summary()

Model: "sequential_9"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_99 (Conv2D)              │ (None, 73, 73, 128)    │        24,704 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_99          │ (None, 73, 73, 128)    │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_100 (Conv2D)             │ (None, 73, 73, 256)    │       819,456 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_100         │ (None, 73, 73, 256)    │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_36 (MaxPooling2D) │ (None, 24, 24, 256)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_101 (Conv2D)             │ (None, 24, 24, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_101         │ (None, 24, 24, 256)    │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_102 (Conv2D)             │ (None, 24, 24, 256)    │        65,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_102         │ (None, 24, 24, 256)    │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_103 (Conv2D)             │ (None, 24, 24, 256)    │        65,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_103         │ (None, 24, 24, 256)    │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_104 (Conv2D)             │ (None, 24, 24, 512)    │     1,180,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_104         │ (None, 24, 24, 512)    │         2,048 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_37 (MaxPooling2D) │ (None, 12, 12, 512)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_105 (Conv2D)             │ (None, 12, 12, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_105         │ (None, 12, 12, 512)    │         2,048 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_106 (Conv2D)             │ (None, 12, 12, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_106         │ (None, 12, 12, 512)    │         2,048 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_38 (MaxPooling2D) │ (None, 6, 6, 512)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_107 (Conv2D)             │ (None, 6, 6, 512)      │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_107         │ (None, 6, 6, 512)      │         2,04

 Total params: 15,609,474 (59.55 MB)

 Trainable params: 15,603,074 (59.52 MB)

 Non-trainable params: 6,400 (25.00 KB)

In [37]:
#normal mobilenet
from tensorflow.keras.callbacks import EarlyStopping

#es = EarlyStopping(monitor='val_accuracy', patience=4, restore_best_weights=True)
es = EarlyStopping(monitor='val_accuracy', patience=4, restore_best_weights=True)
history_custom=custom_model.fit(train, epochs=30, validation_data=val)

Epoch 1/30
   3/1429 ━━━━━━━━━━━━━━━━━━━━ 1:33 66ms/step - accuracy: 0.3194 - loss: 1.5467

W0000 00:00:1714326394.834768     107 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


 322/1429 ━━━━━━━━━━━━━━━━━━━━ 1:02 56ms/step - accuracy: 0.5842 - loss: 1.0344

W0000 00:00:1714326412.898317     106 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


1428/1429 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step - accuracy: 0.6146 - loss: 0.9188

W0000 00:00:1714326460.716114     105 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


1429/1429 ━━━━━━━━━━━━━━━━━━━━ 85s 54ms/step - accuracy: 0.6146 - loss: 0.9186 - val_accuracy: 0.7243 - val_loss: 0.5784
Epoch 2/30
1429/1429 ━━━━━━━━━━━━━━━━━━━━ 71s 49ms/step - accuracy: 0.6680 - loss: 0.6639 - val_accuracy: 0.7439 - val_loss: 0.5484
Epoch 3/30
1429/1429 ━━━━━━━━━━━━━━━━━━━━ 70s 49ms/step - accuracy: 0.7104 - loss: 0.5946 - val_accuracy: 0.7474 - val_loss: 0.5407
Epoch 4/30
1429/1429 ━━━━━━━━━━━━━━━━━━━━ 70s 49ms/step - accuracy: 0.7458 - loss: 0.5315 - val_accuracy: 0.7481 - val_loss: 0.5131
Epoch 5/30
1429/1429 ━━━━━━━━━━━━━━━━━━━━ 72s 50ms/step - accuracy: 0.7759 - loss: 0.4869 - val_accuracy: 0.7768 - val_loss: 0.4714
Epoch 6/30
1429/1429 ━━━━━━━━━━━━━━━━━━━━ 70s 49ms/step - accuracy: 0.8122 - loss: 0.4114 - val_accuracy: 0.7733 - val_loss: 0.4767
Epoch 7/30
1429/1429 ━━━━━━━━━━━━━━━━━━━━ 69s 48ms/step - accuracy: 0.8534 - loss: 0.3492 - val_accuracy: 0.8041 - val_loss: 0.4164
Epoch 8/30
1429/1429 ━━━━━━━━━━━━━━━━━━━━ 70s 49ms/step - accuracy: 0.8999 - loss: 0.26

In [38]:
y_pred_custom = custom_model.predict(test)
pred_custom = np.argmax(y_pred_custom, axis=1) #pick class with highest  probability
labels = (train.class_indices)
labels = dict((v,k) for k,v in labels.items())
pred2_custom = [labels[k] for k in pred_custom]
from sklearn.metrics import confusion_matrix, accuracy_score
y_test = test_images.labels
print(classification_report(y_test, pred2_custom))
print("Accuracy of the Model:","{:.1f}%".format(accuracy_score(y_test, pred2_custom)*100))

  8/447 ━━━━━━━━━━━━━━━━━━━━ 11s 27ms/step

W0000 00:00:1714328506.207616     106 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


447/447 ━━━━━━━━━━━━━━━━━━━━ 16s 33ms/step


W0000 00:00:1714328520.906398     106 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


              precision    recall  f1-score   support

   Bacterial       0.89      0.91      0.90      1146
       Viral       0.84      0.80      0.82       640

    accuracy                           0.87      1786
   macro avg       0.87      0.86      0.86      1786
weighted avg       0.87      0.87      0.87      1786

Accuracy of the Model: 87.3%


In [71]:
custom_model.save("custom_cnn_model.h5")

In [175]:
weight_y_pred_xception = 0.50
weight_y_pred_mobilenet = 0.40
# weight_y_pred_vgg = 0.1
# weight_y_pred_inception = 0
weight_y_pred_custom = 0.10

# ensemble_preds = (weight_y_pred_densenet * y_pred_densenet + weight_y_pred_mobilenet * y_pred_mobilenet + weight_y_pred_vgg * y_pred_vgg + weight_y_pred_inception * y_pred_inception + weight_y_pred_custom * y_pred_custom) / (weight_y_pred_densenet + weight_y_pred_mobilenet + weight_y_pred_vgg + weight_y_pred_inception + weight_y_pred_custom)
# ensemble_preds = np.argmax(ensemble_preds, axis=1)
ensemble_preds = (weight_y_pred_xception * y_pred_xception + weight_y_pred_mobilenet * y_pred_mobilenet + weight_y_pred_custom * y_pred_custom) / (weight_y_pred_xception + weight_y_pred_mobilenet + weight_y_pred_custom)
# ensemble_preds = (weight_y_pred_mobilenet * y_pred_mobilenet + weight_y_pred_custom * y_pred_custom) / (weight_y_pred_mobilenet + weight_y_pred_custom)
ensemble_preds = np.argmax(ensemble_preds, axis=1)

In [176]:
labels = (train.class_indices)
labels = dict((v,k) for k,v in labels.items())
pred2_ensemble = [labels[k] for k in ensemble_preds]

In [177]:
from sklearn.metrics import confusion_matrix, accuracy_score

y_test = test_images.labels
print(classification_report(y_test, pred2_ensemble))
print("Accuracy of the Model:","{:.1f}%".format(accuracy_score(y_test, pred2_ensemble)*100))

              precision    recall  f1-score   support

   Bacterial       0.92      0.95      0.94      1146
       Viral       0.90      0.86      0.88       640

    accuracy                           0.92      1786
   macro avg       0.91      0.90      0.91      1786
weighted avg       0.92      0.92      0.92      1786

Accuracy of the Model: 91.6%
